In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important }</style>"))

In [18]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} .cm-s-ipython span.cm-comment {color: #87CEFA; font-style: italic;}</style>"))

In [5]:
pd.options.display.max_columns = 1000

In [6]:
cust = pd.read_csv("CUST_DATA.csv", encoding="utf-16")
claim = pd.read_csv("CLAIM_DATA.csv", encoding="utf-16")

# 1. 여기부터 전처리 코드를 작성하세요.

## * AGE

In [22]:
cust.AGE = cust.AGE.map(lambda x: int(x//10))

In [24]:
# 전처리한 데이터를 임시 저장
cust.to_csv("CUST_DATA_1-1.csv", index=False, encoding="utf-8-sig")

## * CTPR (지역)
지역 관련 결측치 삭제시 열단위로 삭제하지 않도록 주의

In [25]:
cust = cust.loc[~cust.CTPR.isnull()]

In [30]:
##### 레이블 인코딩
# 지역 텍스트를 숫자로 인코딩 하기 위해, LabelEncoder 클래스를 사용
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(cust.CTPR)
ctpr = le.transform(cust.CTPR)

In [31]:
##### 원-핫 인코딩
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(ctpr.reshape(-1, 1))
ctpr_onehot = enc.transform(ctpr.reshape(-1,1))
ctpr_onehot.toarray()
ctpr_df = pd.DataFrame(ctpr_onehot.toarray(), columns=le.classes_)
# 이렇게 만들어진 ctpr_df 데이터프레임의 인덱스는 0부터 21779까지

# cust_df 데이터프레임은 중간의 행들이 삭제되어서 인덱스가 순서대로 되어있지 않음
# (아까 결측치 삭제해서)
# 따라서, 고객데이터와 원-핫 인코딩한 cust 데이터와 ctpr_df 데이터를 결합(concat)하기 전에,
# cust_df 데이터프레임의 인덱스를 초기화해야 함
cust.reset_index(drop=True, inplace=True)
cust = pd.concat([cust, ctpr_df], axis=1)

In [34]:
# 원본 데이터 열(CTPR)은 삭제
cust.drop("CTPR", axis=1, inplace=True)

In [35]:
cust.head()

,CUST_ID,DIVIDED_SET,SIU_CUST_YN,SEX,AGE,RESI_COST,RESI_TYPE_CODE,FP_CAREER,CUST_RGST,OCCP_GRP_1,OCCP_GRP_2,TOTALPREM,MINCRDT,MAXCRDT,WEDD_YN,MATE_OCCP_GRP_1,MATE_OCCP_GRP_2,CHLD_CNT,LTBN_CHLD_AGE,MAX_PAYM_YM,MAX_PRM,CUST_INCM,RCBASE_HSHD_INCM,JPBASE_HSHD_INCM,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,1,1,N,2,0,21111,20.0,N,199910.0,3.사무직,사무직,146980441.0,NaN,NaN,Y,3.사무직,2차산업 종사자,2.0,13.0,200811.0,319718.0,4879.0,10094,11337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,N,1,0,40000,20.0,N,199910.0,3.사무직,사무직,94600109.0,1.0,6.0,Y,1.주부,주부,2.0,17.0,200012.0,341341.0,6509.0,9143,6509.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,N,1,0,0,NaN,N,199910.0,5.서비스,2차산업 종사자,18501269.0,NaN,NaN,N,NaN,NaN,0.0,0.0,200305.0,131300.0,4180.0,0,4180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,N,2,0,12861,40.0,Y,199910.0,2.자영업,3차산업 종사자,317223657.0,2.0,99.0,N,NaN,NaN,0.0,0.0,201009.0,1493184.0,NaN,4270,5914.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,N,2,0,0,NaN,Y,199910.0,2.자영업,3차산업 종사자,10506072.0,8.0,8.0,Y,3.사무직,고위 공무원,3.0,19.0,200807.0,166760.0,3894.0,0,8885.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-2.csv", index=False, encoding="utf-8-sig")

## * OCCP_GRP_1 (직업 코드)

In [50]:
cust = pd.read_csv("CUST_DATA_1-2.csv", encoding="utf-8-sig")

In [54]:
# map 함수를 이용해서 직업코드 문자열에서 첫 문자만 추출
cust_df.OCCP_GRP_1 = cust_df.OCCP_GRP_1.map(lambda x: str(x)[0])

69

In [55]:
# 전처리한 데이터 임시 저장
cust_df.to_csv("CUST_DATA_1-3.csv", index=False, encoding="utf-8-sig")

69

## * 파생변수 추가
많이 추가할수록 좋다고 하셨음
### 고객별 평균 입원일

In [44]:
# 보험 청구 데이터에서 입원일은 VLID_HOSP_OTDA 열에 저장되어 있음
# 각 고객별로 청구데이터를 분석하여 고객별 평균 입원일을 집계하고, 이를 고객 데이터프레임에 추가해야 함
# 이에 관한 열 이름은 HOSP_DAY
claim_df = pd.read_csv("CLAIM_DATA.csv", encoding="utf-16")

In [ ]:
# 고객의 아이디로 그룹핑 (DataFrameGroupBy 객체)
claim_df_g = claim_df.groupby(claim_df.CUST_ID)

# 그룹핑한 데이터의 평균을 계산하고 VLID_HOSP_OTDA 열만 따로 추출
claim_df_g_mean = claim_df_g.mean()
claim_df_hospday = claim_df_g_mean.loc[:, ["VLID_HOSP_OTDA"]]

# 행 인덱스를 초기화(행 인덱스는 행의 이름이 되도록)
# Result: 고객별 평균입원일을 저장한 데이터프레임
claim_df_hospday.reset_index(level=0, inplace=True)
claim_df_hospday.columns = ['CUST_ID', 'HOSP_DAY']

# 고객 데이터와 고객별 평균입원일을 저장한 데이터프레임을 병합
cust = pd.merge(cust, claim_df_hospday, how="inner")

In [ ]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-4.csv", index=False, encoding="utf-8-sig")

### 사고원인, 청구코드

In [57]:
# 사용자 아이디, 사고원인, 청그코드 데이터 추출
claim_df_acci = claim_df.loc[:, ["CUST_ID", "ACCI_DVSN", "DMND_RESN_CODE"]]
claim_df_acci["value"] = 1

# 피벗 테이블 생성
cust_claim_df = claim_df_acci.pivot_table(index=["CUST_ID"], columns=["ACCI_DVSN", "DMND_RESN_CODE"],
                                         values=["value"], aggfunc='sum', fill_value=0)

cust_claim_df = cust_claim_df.reset_index(level=["CUST_ID"], col_level=1)
cust_claim_df.columns = cust_claim_df.columns.droplevel(level=0)

cust_claim_df.columns = ['_'.join([str(col) for col in cols]) for cols in cust_claim_df.columns]
cust_claim_df.rename(columns={'CUST_ID_':'CUST_ID'}, inplace=True)
cust = pd.merge(cust, cust_claim_df, how="left")

In [ ]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-5.csv", index=False, encoding="utf-8-sig")

# 여기부터는 수정하면 안됩니다.

# 전처리 완료된 파일을 저장하세요. 

In [8]:
cust.to_csv("CUST_DATA_전처리완료.csv", index=False, encoding="utf-8-sig")

In [9]:
cust.shape

(22400, 25)

### 전처리 완료된 데이터를 불러와서 훈련 데이터와 검증데이터로 나눔

In [10]:
cust = pd.read_csv("CUST_DATA_전처리완료.csv", encoding="utf-8-sig")

In [11]:
train = cust[cust['DIVIDED_SET']==1].reset_index(drop=True) # 사기자
test = cust[cust['DIVIDED_SET']==2].reset_index(drop=True) # 사기자가 아닌 사람

In [12]:
train.drop('DIVIDED_SET', axis=1, inplace=True)
test.drop('DIVIDED_SET', axis=1, inplace=True)

In [13]:
train_X = train.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)
train_y = train['SIU_CUST_YN']

In [14]:
test_X = test.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)

In [15]:
train['SIU_CUST_YN'].value_counts()

N    18801
Y     1806
Name: SIU_CUST_YN, dtype: int64

In [16]:
train_X.shape, train_y.shape

((20607, 22), (20607,))

### Random Forest를 이용한 모델링

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_features=16, random_state=42)
rf_model.fit(train_X, train_y)

ValueError: could not convert string to float: 'N'

In [ ]:
y_pred = rf_model.predict(test_X)

In [ ]:
y_pred

In [ ]:
val_df = pd.read_csv("answer.csv") # 모평을 평가하기 위한 정답을 불러옴

In [170]:
val_df.head()

,CUST_ID,YN
0,37,0
1,51,0
2,60,0
3,65,0
4,73,0


In [171]:
pd.crosstab(val_df.YN, y_pred) # 교차분류표

col_0,0.0,1.0
YN,,
0,1571,64
1,152,6


In [172]:
rf_model.score(test_X, val_df.YN)

0.879531511433352

### 10회 교차검증 후 f1 스코어의 평균을 출력합니다. 
### 출력한 값이 가장 큰 팀이 우승

In [173]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf_model, test_X, val_df.YN , cv=10, scoring='f1_macro')
scores.mean()

0.47680068420348815